# Create a PDF inheriting from RooHSEventsPDF
In general for RooFit the PDF must be a derived class of RooAbsPdf

Often for fits to experimental data we must include experimental acceptances in the distribution. This is done in the extended maximum likelhood technique through convoluting the acceptance in the normalisation integral caclulation. In practise this done via summation of the PDF function over accepted simulated events from flat distributions (in the fit variables).

This is incorporated into the RooHSEventsPDF class and so we will inherit directly from it. Currently there is no actual acceptance effects in this example, but we include possibility to use the simulated integration to show how it works.

Reminder, from the GenerateData notebook the azimuthal distribution had the form

    f(#phi) = 1 + A h P cos(2#phi) - B h P sin(2#phi)   --(1)
Where I have now introduced the parameters A and B instead of the numerical values. I have 3 experimentally measured quantities, #phi, h and P . In the data tree these correspond to Phi, PolState and Pol respectively.

### Create the PDF class code

This is done via a skeleton code maker, which we must first load :

In [ ]:
import ROOT
ROOT.gROOT.ProcessLine(".x $BRUFIT/macros/LoadBru.C")
ROOT.gROOT.ProcessLine(".L RooFitSkeleton.cpp+")

Now we declare the PDF with 3 observables #phi,PolState (Note it is a category==int, so need CAT:) and Pol and 2 fit parameters A and B.

In [ ]:
skel = ROOT.RooFitSkeleton()
skel.CreateRooFitEventsPDF("PhiAsymmetry","Phi,Pol,CAT:PolState","A,B")

Now I have the code I must edit it to include the actual functional form in the evaluate and evaluateMC functions.

These are at the bottom of the file. I can open files in notebooks using the %edit command.

Please note in the evaluateMC function the variables phi, Pol and PolState are prefixed with mc, but it should have exactly the same form as evaluate. For a PDF of the form of eqn. (1) I use must implement the following evaulate functions

     Double_t PhiAsymmetry::evaluate() const { 
                 return 1.0 + PolState*Pol*(A*TMath::Cos(2*TMath::DegToRad()*Phi)+B*TMath::Sin(2*TMath::DegToRad()*Phi));
     } 

    Double_t PhiAsymmetry::evaluateMC(const vector<Float_t> *vars,const  vector<Int_t> *cats) const {
      Double_t mcPhi=(*vars)[fTreeEntry*fNvars+0];
      Double_t mcPol=(*vars)[fTreeEntry*fNvars+1];
      Int_t mcPolState=(*cats)[fTreeEntry*fNcats+0];

      return 1.0 + mcPolState*mcPol*(A*TMath::Cos(2*TMath::DegToRad()*mcPhi)+B*TMath::Sin(2*TMath::DegToRad()*mcPhi));
    }
    
You should be able to copy these into your PhiAsymmetry.cxx code.

You should be able to open this from Jupyter home. In principle %edit should open a file for editting, but it does not work for me here. It does in the ROOTbook version CreatePDFC.

In [ ]:
%edit PhiAsymmetry.cxx

In [ ]:
ls